# Planet Download Example
This notebook shows how to download planet imagery uploaded to a private s3 bucket for OPERA DSWx validation

In [1]:
import geopandas as gpd
import boto3
import os

### AWS Credentials
In order to download imagery from the private bucket, JPL RSA access and OPERA Calval AWS credentials are needed

In [2]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

### S3 Buckets
A table containing image metadata is located on a public s3 bucket, but the commercial image files can't be openly shared and are stored in a seperate private bucket. S3 keys (path within a bucket) that point to the image file locations are stored in the metadata table in the public bucket

In [3]:
bucket_name = 'opera-calval-database-dswx'
private_bucket_name = 'opera-calval-database-dswx-private'

### Read Image metadata table
Image metadata is stored in a geojson table, which can be read as a geodataframe using geopandas. This dataframe can be spatial searched or searched by metadata fields

In [4]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
images = gpd.read_file(image_table_data)
images.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
0,0.0,HLS.S30.T10VEK.2021268T192141.v2.0,2021-09-25 19:17:08,20210925_191708_94_2408,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/3_42/20210925_191708_94_2408/20210925_191...,84.975663,3_42,0.007904,"POLYGON ((-122.94132 57.97664, -123.03517 57.7..."
1,0.0,HLS.L30.T18UXG.2021300T154817.v2.0,2021-10-27 15:14:45,20211027_151445_21_2231,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_21/20211027_151445_21_2231/20211027_151...,52.216780,4_21,0.023288,"POLYGON ((-71.93982 55.28796, -72.02213 55.104..."
2,0.0,HLS.S30.T18FYH.2021302T141741.v2.0,2021-10-30 14:26:13,20211030_142613_41_227b,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/2_8/20211030_142613_41_227b/20211030_1426...,74.756870,2_8,0.992265,"POLYGON ((-71.22036 -51.94097, -71.30174 -52.1..."
3,0.0,HLS.S30.T36LWH.2021268T073641.v2.0,2021-09-26 07:24:14,20210926_072414_71_2233,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_3/20210926_072414_71_2233/20210926_0724...,97.600689,4_3,0.972010,"POLYGON ((33.42779 -16.15910, 33.38993 -16.342..."
4,0.0,HLS.S30.T15VUK.2021244T175911.v2.0,2021-09-01 17:52:05,20210901_175205_71_2426,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_34/20210901_175205_71_2426/20210901_175...,96.416886,4_34,0.009994,"POLYGON ((-96.94381 62.88876, -97.07609 62.707..."


### Download Function
This function takes a single row of the image geodataframe and a download location (folder path on local computer) and downloads the image file associated with the provided row

In [5]:
def downloadImage(row,downloadDir):
    bucket = row.s3_bucket.iloc[0]
    key = row.s3_key_image.iloc[0]
    image_name = row.image_name.iloc[0]
    response = s3_client.download_file(bucket,key,downloadDir+image_name)

### Basic Usage Example
This example extracts the first image from the image table and downloads it to a sub directory within the current working directory called "downloads".

In [6]:
downloadDir = os.path.expanduser('./downloads/')
os.makedirs(downloadDir, exist_ok=True) 

In [7]:
site_name_4download='4_37' #Select Planet image by UMD GLAD chip number
search = images[images.site_name == site_name_4download] 
search.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
8,0.0,HLS.S30.T35LPK.2021267T080651.v2.0,2021-09-24 08:20:25,20210924_082025_48_2424,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_37/20210924_082025_48_2424/20210924_082...,99.136845,4_37,0.008675,"POLYGON ((28.20956 -9.14523, 28.17025 -9.34005..."


In [8]:
row_indx=search[search['site_name'] == site_name_4download].index[0]
row = images.iloc[[row_indx]] 

In [9]:
downloadImage(row,downloadDir)